#Import Libraries

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter # to calculate word frequencies
from itertools import product # Creating a combination for grid search

# data loading and preprocessing

In [64]:
# product reviews
text = """ Bu ürün beklentimi fazlasıyla karşıladı!
Malzeme kalitesi, işçilik... hepsi beklediğimden iyi.
Kutu içeriği eksiksizdi; kurulumu da oldukça kolaydı.
Performansı günlük kullanımda çok akıcı, sessiz ve stabil.
Kısacası: bu fiyata kesinlikle çok iyi tekrar tercih ederim! """

# data preprocessing:
# get rid of punctuation.
# convert to lowercase.
# split words

words = text.replace('.', '').replace(',', '').replace('!', '').replace(':', '').replace(';', '').lower().split()

# calculate word frequencies and create indexes

word_count = Counter(words)
vocab = sorted(word_count, key=word_count.get, reverse=True) # sort word frequency from smallest to largest
word_2_index = {word: i for i, word in enumerate(vocab)}
index_2_word = {i: word for i, word in enumerate(vocab)}

# training data preparation

data = [(words[i],words[i+1]) for i in range(len(words) - 1)]


# lstm model definition

In [65]:
class LSTM(nn.Module):

  def __init__(self, vocab_size, embedding_dim, hidden_dim):

      super(LSTM, self).__init__() # calling a superclass's constructor
      self.embedding = nn.Embedding(vocab_size, embedding_dim) # embedding layer
      self.lstm = nn.LSTM(embedding_dim, hidden_dim) # lstm layer
      self.fc = nn.Linear(hidden_dim, vocab_size) # fully connected layer

  def forward(self, x): # feed forward function

      x = self.embedding(x) # embedding layer
      lstm_out, _ = self.lstm(x.view(1,1,-1)) # lstm layer
      output = self.fc(lstm_out.view(1,-1)) # fully connected layer
      return output

model = LSTM(len(vocab), embedding_dim = 8, hidden_dim = 32)




# hyperparameter tuning

In [66]:
# word list -> tensor
def prepare_sequence(seq, to_ix):
    return torch.tensor([to_ix[w] for w in seq], dtype=torch.long)

# Let's determine the hyperparameter tuning combinations
embedding_sizes = [8, 16] # Embedding sizes to try
hidden_sizes = [32, 64] # hidden layer sizes to be tested
learning_rates = [0.01, 0.005] # learning rate

best_loss = float('inf') # to keep the loss value to a minimum
best_params = {} # to keep the parameters that give the lowest loss value

print("Hyperparameter Tuning")

# grid search
for emb_size, hidden_size, lr in product(embedding_sizes, hidden_sizes, learning_rates):
    print(f"Attempt: Embedding Size: {emb_size}, Hidden Size: {hidden_size}, Learning Rate: {lr}")

    # modeli tanımla
    model = LSTM(len(vocab), emb_size, hidden_size) # create model with selected parameters

    # Define optimizer and loss function
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    epochs = 100
    total_loss = 0
    for epoch in range(epochs):
        epoch_loss = 0 # reset loss at epoch start
        for word, next_word in data:
            model.zero_grad() # reset gradients
            input_tensor = prepare_sequence([word], word_2_index) # create input tensor
            target_tensor = prepare_sequence([next_word], word_2_index) # create output tensor
            output = model(input_tensor) # model çıktısını al
            loss = loss_function(output, target_tensor)
            loss.backward() # apply backpropagation
            optimizer.step() # update parameters
            epoch_loss += loss.item()

        if epoch % 10 == 0:
            print(f"Epoch: {epoch}, Loss: {epoch_loss:.5f}")
        total_loss = epoch_loss

    # save best model
    if total_loss < best_loss:
        best_loss = total_loss
        best_params = {'embedding_dim': emb_size, 'hidden_dim': hidden_size, 'learning_rate': lr}
    print()

print("Best Parameters:", best_params)




Hyperparameter Tuning
Attempt: Embedding Size: 8, Hidden Size: 32, Learning Rate: 0.01
Epoch: 0, Loss: 121.90928
Epoch: 10, Loss: 8.70070
Epoch: 20, Loss: 6.51993
Epoch: 30, Loss: 6.00152
Epoch: 40, Loss: 5.73674
Epoch: 50, Loss: 5.60921
Epoch: 60, Loss: 5.46220
Epoch: 70, Loss: 5.37964
Epoch: 80, Loss: 5.31057
Epoch: 90, Loss: 5.24786

Attempt: Embedding Size: 8, Hidden Size: 32, Learning Rate: 0.005
Epoch: 0, Loss: 119.42760
Epoch: 10, Loss: 17.19873
Epoch: 20, Loss: 7.16656
Epoch: 30, Loss: 6.00794
Epoch: 40, Loss: 5.59127
Epoch: 50, Loss: 5.37616
Epoch: 60, Loss: 5.24624
Epoch: 70, Loss: 5.16843
Epoch: 80, Loss: 5.10373
Epoch: 90, Loss: 5.05018

Attempt: Embedding Size: 8, Hidden Size: 64, Learning Rate: 0.01
Epoch: 0, Loss: 121.12013
Epoch: 10, Loss: 7.73745
Epoch: 20, Loss: 6.48762
Epoch: 30, Loss: 5.96138
Epoch: 40, Loss: 5.79164
Epoch: 50, Loss: 5.56153
Epoch: 60, Loss: 5.43906
Epoch: 70, Loss: 5.35139
Epoch: 80, Loss: 5.20299
Epoch: 90, Loss: 5.20615

Attempt: Embedding Size: 

#lstm training

In [67]:
final_model = LSTM(len(vocab), best_params['embedding_dim'], best_params['hidden_dim'])
optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
loss_function = nn.CrossEntropyLoss() # entropy loss function

print("Final model training")
epochs = 100
for epoch in range(epochs):
    epoch_loss = 0
    for word, next_word in data:
        final_model.zero_grad()
        input_tensor = prepare_sequence([word], word_2_index)
        target_tensor = prepare_sequence([next_word], word_2_index)
        output = final_model(input_tensor)
        loss = loss_function(output, target_tensor)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Final Model Epoch: {epoch}, Loss: {epoch_loss:.5f}")


Final model training
Final Model Epoch: 0, Loss: 119.23145
Final Model Epoch: 10, Loss: 16.84916
Final Model Epoch: 20, Loss: 7.16482
Final Model Epoch: 30, Loss: 6.00450
Final Model Epoch: 40, Loss: 5.58995
Final Model Epoch: 50, Loss: 5.38060
Final Model Epoch: 60, Loss: 5.25336
Final Model Epoch: 70, Loss: 5.16809
Final Model Epoch: 80, Loss: 5.10590
Final Model Epoch: 90, Loss: 5.07441


# testing and evaluation

In [72]:
# word prediction function: initial development and generate n words
def predict_sequence(start_word, num_words):
    current_word = start_word # the current word is set as the starting word
    output_sequence = [current_word] # output sequence

    for _ in range(num_words): # guess the specified number of words
        with torch.no_grad(): # without performing gradient calculation
            input_tensor = prepare_sequence([current_word], word_2_index) # tensor transform from word
            output = final_model(input_tensor) # call model
            predicted_index = torch.argmax(output).item() # get the index of the word with the highest probability
            predicted_word = index_2_word[predicted_index] # returns the word corresponding to the index
            output_sequence.append(predicted_word)
            current_word = predicted_word # update current words for next prediction

    return output_sequence # return the predicted word string

"""
Bu ürün beklentimi fazlasıyla karşıladı!
Malzeme kalitesi, işçilik... hepsi beklediğimden iyi.
Kutu içeriği eksiksizdi; kurulumu da oldukça kolaydı.
Performansı günlük kullanımda çok akıcı, sessiz ve stabil.
Kısacası: bu fiyata kesinlikle çok iyi tekrar tercih ederim!
"""



start_word = "ürün"
num_predictions = 3
predict_sequence = predict_sequence(start_word, num_predictions)
print(" ".join(predict_sequence))



ürün beklentimi fazlasıyla karşıladı
